In [42]:
import torch
import torch.nn.functional as F

from torch_geometric.nn import GCNConv, VGAE
from torch_geometric.loader import DataLoader

from torchvision import transforms

from lib.lib import SignatureDataset, image_to_graph

from tqdm import tqdm

In [43]:
def transform(**kwargs):
    return transforms.Compose([
        transforms.Grayscale(num_output_channels=kwargs['num_output_channels']),
        transforms.Resize(kwargs['resize']),
        transforms.ToTensor(),
    ])

dataset = SignatureDataset(
    root_dir="test_image",
    transform=transform(num_output_channels=1, resize=(150, 150))
)

Loaded 3 signature images (genuine + forged)


In [44]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, latent_dim):
        super(GNNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv_mu = GCNConv(hidden_channels, latent_dim)
        self.conv_logvar = GCNConv(hidden_channels, latent_dim)

    def forward(self, x, edge_index):
        # Step 1: Aggregate node features from neighbors
        x = F.relu(self.conv1(x, edge_index))

        # Step 2: Output mean and log variance
        mu = self.conv_mu(x, edge_index)
        logvar = self.conv_logvar(x, edge_index)

        return mu, logvar

In [45]:
test_graph = []

# Convert training dataset
for t in tqdm(dataset, desc="Train Graphs", leave=False):
    t_graph = image_to_graph(t)
    test_graph.append(t_graph)

print("Train graphs:", len(test_graph))

Train graphs: 3


In [63]:
test_loader = DataLoader(
    test_graph,
    batch_size=3,        # Process 32 graphs at once
    shuffle=True,         # Shuffle for training
    num_workers=0,        # 0 for debugging, 2-4 for faster loading
    # pin_memory=True 
)

next(iter(test_loader))

DataBatch(x=[3072, 3], edge_index=[2, 11904], batch=[3072], ptr=[4])

In [64]:
input_dim = next(iter(test_graph)).x.shape[1]
hidden_dim = 64
latent_dim = 128

In [65]:
model = VGAE(GNNEncoder(in_channels=input_dim, hidden_channels=hidden_dim, latent_dim=latent_dim))
model.load_state_dict(torch.load('VGAE_Model.pt', weights_only=True))
model.eval()

VGAE(
  (encoder): GNNEncoder(
    (conv1): GCNConv(3, 64)
    (conv_mu): GCNConv(64, 128)
    (conv_logvar): GCNConv(64, 128)
  )
  (decoder): InnerProductDecoder()
)

In [79]:
data = test_graph[0]
data

Data(x=[1024, 3], edge_index=[2, 3968])

In [80]:
with torch.no_grad():
    z = model.encode(data.x, data.edge_index)

z

tensor([[ 0.0464,  0.0318,  0.0354,  ..., -0.1222,  0.0228,  0.0388],
        [ 0.0478,  0.0379,  0.0339,  ..., -0.1361,  0.0277,  0.0396],
        [ 0.0499,  0.0502,  0.0263,  ..., -0.1482,  0.0333,  0.0392],
        ...,
        [-0.0979, -0.1036, -0.3266,  ...,  0.1507,  0.1983,  0.2764],
        [-0.1074, -0.1091, -0.3531,  ...,  0.1390,  0.2204,  0.2983],
        [-0.0948, -0.0944, -0.3250,  ...,  0.1040,  0.2092,  0.2797]])